In [100]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [101]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [102]:
#preprocess the data  (rowNumber,CustomerId,Surname is of no use so delete(drop) those column from the dataset)
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1, errors='ignore')
data


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [103]:
# Encode categorical variables-->for gender(as only male and female present so apply label_encoder)
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [104]:
# One-hot encode 'Geography'
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [105]:
# Combine one-hot encoded columns with original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
# Split the data into features and target(we have to predict Estimated salary so we are sending it to y )
y = data['EstimatedSalary']
X = data.drop(['EstimatedSalary','Exited'], axis=1)


In [107]:
## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [108]:
## Scale these features-->convert it into array
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
# Save the encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


##ANN REGRESSION PROBLEM STATEMENT

In [110]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [111]:
# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # here we didnot apply any activation funcyion if we dont apply any activation function then default activation function will be apply which is linear activation function
])

## compile the model (for loosing the error)
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])   #Optimizer decides HOW the model updates its weights-->It automatically adjusts the learning rate,Loss function tells the model how wrong it is,metric is same as loss, so you’ll see MAE in output.

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 64)                832       
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [112]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up TensorBoard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")  #adds current date & time to folder name,
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)  #Basically, this watches your model during training.

In [113]:
# Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)  #monitor='val_loss-->This tells the callback which metric to watch during training


In [114]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 100364.2344 - mae: 100364.2344 - val_loss: 98463.9688 - val_mae: 98463.9688
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 99402.2188 - mae: 99402.2188 - val_loss: 96518.0078 - val_mae: 96518.0078
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 96009.7344 - mae: 96009.7344 - val_loss: 91599.9766 - val_mae: 91599.9766
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 89456.2344 - mae: 89456.2344 - val_loss: 83610.4766 - val_mae: 83610.4766
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 80255.5469 - mae: 80255.5469 - val_loss: 73944.2266 - val_mae: 73944.2266
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 70200.5859 - mae: 70200.5859 - val_loss: 64722.7422 - val_mae: 64722.7422
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 61454.2969 - mae: 61454.2969 

In [116]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [117]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 27744), started 21:49:03 ago. (Use '!kill 27744' to kill it.)

In [121]:
#Evaluate model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE:{test_mae}')

63/63 [==============================] - 0s 814us/step - loss: 50352.7109 - mae: 50352.7109
Test MAE:50352.7109375


In [122]:
model.save('regression_model.h5')